# CART Decision Tree for Optical Character Recognition

Here, we build a decision tree to classify between two handwritten numbers, 3 and 5. The data set used is from Oregon State University, Fall 2018 CS 534 Machine Learning Course Implementation Assignment 3.<br><br>
There are 100 features for each sample, corresponding to the top 100 principal components generated using PCA from the original sample features of size 28 * 28.<br><br>
The training set contains 4888 samples. Each sample is a list of 101 values. The first column represents the digit’s label  which is 3 or 5. The other 100 floating values are the PCA-generated features for this sample.<br><br>
The validation set contains 1629 samples. This set will be used to tune the hyper parameters and select the best model.<br>
No test set is provided.<br>

# Import Libraries

## Preprocessing
Decision Tree algorithm does not need any scaling of the features.

In [161]:
import copy
import time
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline


from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [162]:
class Preprocessing:
    
    def __init__(self, raw_training_data_path, raw_validation_data_path):
            
            # Read the training and validation data. As the features are just PCA components, no need to observe the 
            # column names, so can proceed with numpy instead of pandas.
            
            self.raw_training_data = np.genfromtxt(raw_training_data_path, delimiter = ",")
            self.raw_validation_data = np.genfromtxt(raw_validation_data_path, delimiter = ",")
            
            print(self.raw_training_data[0:10,:])
            
            # split training features and labels
            self.training_features = self.raw_training_data[:, 1:]
            self.training_labels = self.raw_training_data[:, 0]
            
            print("No of training samples: {}, No of training features: {}".format(len(self.training_labels), self.training_features.shape[1]))
            
            
            # split validation features and labels
            self.validation_features = self.raw_validation_data[:, 1:]
            self.validation_labels = self.raw_validation_data[:, 0]
            
            print("No of validation samples: {}".format(len(self.validation_labels), ))

        
        

# Decision Tree Implementation with GINI Impurity

In [163]:
class Decision_Tree_Node:
    
    def __init__(self, X = None, y = None, depth = None):
        self.X = X
        self.y = y

        self.left = None
        self.right = None
        self.depth = depth
        self.feature_index_with_optimal_split = None
        self.threshold_for_optimal_split = None # since features are real values
        self.is_leaf = False
        self.class_probabilities = None


class Decision_Tree(BaseEstimator, ClassifierMixin):
    
    '''
    Implementation of CART Decision Tree.
    
    Parameters
    -------------
    max_depth : int, optional, default 100
                The maximum depth of the deicison tree for early-stopping/ to avoid overfitting to outliers.
    min_pool :  int, optional, default 10
                The minimum number of samples at a node after split for early-stopping/ to avoid overfitting.
    '''
    
    
    
    def __init__(self, root, max_depth = None, min_pool = None,):
        
        self.root = root
        self.max_depth = max_depth
        self.min_samples = min_pool
        self.classes = None
        
        
        
    
    
    def build_decision_tree(self, X, y, node, current_depth):

        '''
        Recursively builds the decision tree if the terminal conditions not met
        
        Parameters
        -----------------
        X : training features
        y : training labels
        '''
        
        
        
        
        
        
        # base cases of recursion
        if current_depth >= self.max_depth:
            node.is_leaf = True
            
            
        elif len(y) <= self.min_samples:
            node.is_leaf = True
            
        
        elif len(np.unique(y)) == 1:
            node.is_leaf = True
        
        else:
            
            
            
            # find the best split and the feature that gives this best split
            feature_for_best_split, best_split_threshold, X_left, X_right, y_left, y_right = self.best_split(X, y)#, node.depth) 
            
            
            if feature_for_best_split is None:
                # best_split() returns None if no feature gave a split that has less impurity than the parent
                node.is_leaf = True
                
            else:
                
                # Recursively build trees to the left and the right using the best features and the best split threshold
                node.depth = current_depth
                node.feature_index_with_optimal_split = feature_for_best_split
                node.threshold_for_optimal_split = best_split_threshold
                
                node.left = Decision_Tree_Node(X_left, y_left, current_depth+1)
                node.right = Decision_Tree_Node(X_right, y_right, current_depth+1)
                
                # when a node is encountered, calculate the class probabilities for each class at this node and store
                node.left.class_probabilities = self.calculate_class_probabilities(y_left)
                node.right.class_probabilities = self.calculate_class_probabilities(y_right)
                
                self.build_decision_tree(X_left, y_left, node.left, current_depth+1)
                self.build_decision_tree(X_right, y_right, node.right, current_depth+1)
            
        
    
        
    def gini_data(self,y): #####################MAKE THIS EFFICEINT
        #count the Y equal to one
        c_root_pos=np.count_nonzero(y==3) # The labels are 3 and 5, 3 is the first label in ascending order.
        c_root_total = len(y)        
        
        gini_of_y =  1- (c_root_pos/c_root_total)**2 -((c_root_total-c_root_pos)/c_root_total)**2 
        return gini_of_y
    
    def gini_after_split(self, y_copy, indices_where_change, i):
        
        '''
        Calculate GINI impurity metric U after split
        
        U = (p^2) + (q^2 )
        
        where p = No of samples at this node with feature x below the node threshold/ No of samples at this node
              q = No of samples at this node with feature x equal or above the node threshold/ No of samples at this node
        
        Parameters
        -----------------
        '''
        
        CL_pos = np.count_nonzero(y_copy[:indices_where_change[i] + 1] == 3)
        CL_total = len(y_copy[:indices_where_change[i] + 1])
        CL_neg = CL_total - CL_pos

        # calculate gain
        prob1 = CL_pos / (CL_total)#sample_num+1/len(y)
        prob2 = CL_neg / (CL_total)#(len(y) - (sample_num+1) )/len(y)

        UAL = 1 - (prob1) ** 2 - (prob2) ** 2



        CR_pos = np.count_nonzero(y_copy[indices_where_change[i] + 1:] == 3)
        CR_total = len(y_copy[indices_where_change[i] + 1:])
        CR_neg = CR_total - CR_pos
        prob1 = CR_pos / (CR_total)
        prob2 = CR_neg / (CR_total)
        UAR = 1 - (prob1) ** 2 - (prob2) ** 2


        pl = (CL_total) / len(y_copy)
        pr = (CR_total) / len(y_copy)
        
        gini_after_split =  pl * UAL + pr * UAR
        return gini_after_split
    
    
    
    def split_node(self, X, y, best_feature, max_benefit_val):
    
        left_X, right_X, left_y, right_y = [],[],[],[]

        for row in range(0,X.shape[0]):
            if X[row][best_feature] < max_benefit_val:
                left_X.append(X[row])
                left_y.append(y[row])
            else:
                right_X.append(X[row])
                right_y.append(y[row])
        
        return np.array(left_X), np.array(right_X), np.array(left_y), np.array(right_y)
        
        
        
        
        
    def best_split(self, X_passed, y_passed):
        '''
        Divides the training samples into optimal splits according to the impurity metric
        '''
        numpy_X = []
        numpy_y = []
        
        for row in range(0,len(X_passed)):
            numpy_X.append(X_passed[row])
            numpy_y.append(y_passed[row])
        
        X = np.array(numpy_X)
        y = np.array(numpy_y)
        
        
        gini_before_split = self.gini_data(y)

        benefit_of_split = 0
        
        benefit_vals_for_one_feature = np.zeros(X.shape[1])
        max_benefit_index_for_each_feature = np.zeros(X.shape[1])

        
        # for each feature, find the GINI gain at each threshold
        for feature in range(0,X.shape[1]):

            
            
            
            current_feature = X[:, feature]
            
            
            
            
            # TRICK: Sort the training samples according to this feature's values
            # and find GINI only at samples where the class label changes.
            
            
            stacked = np.column_stack((y, current_feature)) # stack y_to_take and current_feature
            sortedd = stacked[np.argsort(stacked[:, 1])]
            
            y_copy = sortedd[..., 0]
            #indices_where_change = np.where(np.roll(y_copy,1)!=y_copy)[0]

            indices_where_change = np.where(y_copy[:-1] != y_copy[1:])[0]
                                            
            for i in range(len(indices_where_change)):

                gini_after_split = self.gini_after_split(y_copy, indices_where_change, i)

                benefit_of_split = gini_before_split - gini_after_split


                if benefit_of_split >  benefit_vals_for_one_feature[feature]:
                    benefit_vals_for_one_feature[feature] = benefit_of_split  
                    max_benefit_index_for_each_feature[feature] = indices_where_change[i]
                   

                
                    

        if benefit_of_split == 0:
            #print("No good feature found.")
            return None, None, None, None, None, None
        else:
            
            best_feature = np.argmax(benefit_vals_for_one_feature)
            index_of_threhold_with_max_benefit_for_best_feature = max_benefit_index_for_each_feature[best_feature]
            
            # lets get the column of that feature and sort and get the threshold
            max_benefit_feature = copy.deepcopy(X[:,best_feature])
            max_benefit_feature.sort()
            
            # Athough we have threshold_index_for_best_split_in_indices_where_change, we need to find the
            # corresponding index in the sorted max benefit feature.
            

            max_benefit_val = max_benefit_feature[int(index_of_threhold_with_max_benefit_for_best_feature)]
            
            
            X_left, X_right, y_left, y_right = self.split_node(X, y, best_feature, max_benefit_val)
            
            

                
                
            
            
                        
            # feature_for_best_split, best_split_threshold, X_left, X_right, y_left, y_right
            return best_feature, max_benefit_val, X_left, X_right, y_left, y_right
                
                        

    def calculate_class_probabilities(self, y):
        '''
        Calculates the probability of each class at this node
        '''
        
        class_probabilities = []
        
        for label in self.root.classes:
            
            if len(y[y==label]) == len(y) :
                class_prob = 1
            elif len(y[y==label]) == 0:
                class_prob = 0
            else:
                class_prob = y[y==label].shape[0]/len(y)
            
            class_probabilities.append(class_prob)
        
        return class_probabilities
    
    
    
    
    def calculate_gini_impurity(self, labels):
        '''
        Calculate GINI impurity metric U at a node of the decision tree
        
        U = 1 - [(p^2) + (q^2 )]
        
        where p = No of samples at this node with feature x below the node threshold/ No of samples at this node
              q = No of samples at this node with feature x equal or above the node threshold/ No of samples at this node
        
        Parameters
        -----------------
        
        
        '''
        probabilities = []
        for class_label in self.root.classes:
            probability_of_class = len(labels[labels == class_label])/ len(labels)
            probabilities.append(probability_of_class)
        
        impurity = 1 - sum([pow(elem,2) for elem in probabilities])
        assert impurity <= 1
        
        return impurity
        
            
            
    
    def predict_a_sample(self, x, node):
        '''
        Predicts a class label for the passed test sample by recursively calling a child node from the root upto
        a leaf node of the learned decision tree.
        
        Parameters
        ------------------
        x : a single test sample features
        node : a node of the learned decision tree
        '''
        
        if node.is_leaf:
            return node.class_probabilities
        else:
            
            if x[node.feature_index_with_optimal_split] < node.threshold_for_optimal_split:
                return self.predict_a_sample(x, node.left)
            else:
                return self.predict_a_sample(x, node.right)
    
    
    
    def predict(self, X):
        '''
        Predicts class labels for all test samples passed.
        
        Parameters
        ---------------------------
        X : test samples passed as 2D arrays
        '''
        
        
        
        test_preds = []
        for test_sample in X:
            class_probabilities = self.predict_a_sample(test_sample, self.root)
            test_pred = np.argmax(class_probabilities)
            if test_pred == 0:
                
                test_preds.append(3)
            elif test_pred == 1:
                test_preds.append(5)
            else:
                raise AssertionError
        
        return test_preds
        
    
        
        
        
        
        
    
    def fit(self, X, y):
        
        '''
        Train CART decision tree using GINI impurity.
        
        Parameters
        ----------------
        X : arraylike
            The training samples in 2D format, corresponding to input features of each sample.
        y : arraylike, optional, default None
            The labels for each sample in X. Some algorithms do not require the label y while some (like Decisoin Tree) do.
            Hence, keep y as optional.
        
        Returns
        ----------------
        self : Trained Decision Tree object
               This method returns self for compatibility reasons with sklearn's other interfaces/ functionalities.
        
        
        '''
        self.root = Decision_Tree_Node(X, y, 0)
        self.root.classes = [3., 5.]#list(set(y)) # gives the set of unique elements in y.
        self.root.classes.sort() # sorts in ascending order in-place if not already sorted
        
        print("The unique classes present in training data:{}".format(self.root.classes))
        
        self.build_decision_tree(X = X, y = y, node = self.root, current_depth = 0)
    
        
        
        

# Execution

In [164]:
# read data
training_data_path = "data/pa3_train_reduced.csv"
validation_data_path = "data/pa3_valid_reduced.csv"

preprocessing = Preprocessing(training_data_path, validation_data_path)




[[   5.     -252.5092 1040.6188 ...  -21.6094  -32.602    25.619 ]
 [   5.     -684.5502 -368.7191 ...  -36.3467   26.6937  -17.564 ]
 [   3.      119.2934  638.9885 ...   14.7913   48.7926  -94.5664]
 ...
 [   5.      972.0162   77.9232 ...  -35.5166  -16.6162  -43.1298]
 [   5.     -255.5209  144.6523 ...   75.572    34.6369   24.2973]
 [   3.      217.1434  592.4619 ...  -25.7318   55.1806    4.9309]]
No of training samples: 4888, No of training features: 100
No of validation samples: 1629


### Training Data Analysis
We can try and see if there exists just one feature that has good threholds that separates different classes in the pairplots. 

In [165]:
for col in range(1,len(preprocessing.training_features[0])):
    feature_and_target = np.stack((  preprocessing.training_features[:,0], preprocessing.training_features[:,col] ), axis = 1)
    #sns.pairplot(pd.DataFrame(feature_and_target))


In [166]:
# train
root = Decision_Tree_Node(0)
decision_tree = Decision_Tree(root, max_depth = 20, min_pool = 1)


decision_tree.fit(preprocessing.training_features, preprocessing.training_labels)

The unique classes present in training data:[3.0, 5.0]


In [167]:
# predict for both training data and test data
training_predictions = decision_tree.predict(preprocessing.training_features)
test_predictions = decision_tree.predict(preprocessing.validation_features)

In [168]:
# predict on training and test data
confusion_matrix(preprocessing.training_labels, training_predictions)

array([[2572,   49],
       [  78, 2189]], dtype=int64)

In [169]:
accuracy_score(preprocessing.training_labels, training_predictions)

0.9740180032733224

In [170]:
confusion_matrix(preprocessing.validation_labels, test_predictions)

array([[830,  48],
       [ 90, 661]], dtype=int64)

In [171]:
accuracy_score(preprocessing.validation_labels, test_predictions)

0.9152854511970534